In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, transform
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.datasets import mnist

## Loading the dataset

In [4]:
train_data = pd.read_csv("input/train.csv")
test_data = pd.read_csv("input/test.csv")
print(train_data.shape, test_data.shape)

(42000, 785) (28000, 784)


## Loading the MNIST Dataset

In [5]:
(x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()
x_train1 = np.concatenate((x_test1, x_train1))
y_train1 = np.concatenate((y_test1, y_train1))

x_train1 = x_train1.reshape((x_train1.shape[0], 28, 28, 1))
print(x_train1.shape, y_train1.shape)

(70000, 28, 28, 1) (70000,)


### Preparing the data

In [6]:
x = np.array(train_data.drop(['label'], axis = 1))
y = np.array(train_data['label'])
test_data = np.array(test_data)

x = x.reshape((x.shape[0], 28, 28, 1))
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

x = np.concatenate((x, x_train1))
y = np.concatenate((y, y_train1))

x = x/255
test_data = test_data/255
y = to_categorical(y, num_classes = 10)

print(x.shape, y.shape)

(112000, 28, 28, 1) (112000, 10)


### Spliting the dataset into training and testing set

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, shuffle = True)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(100800, 28, 28, 1) (100800, 10) (11200, 28, 28, 1) (11200, 10)


# Creating a deep neural network

In [8]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D((2,2)))

model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))

model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.30))
model.add(Dense(10, activation = "softmax"))


optimizer = RMSprop(learning_rate = 0.01, rho = 0.9, epsilon = 1e-08, decay = 0.0)

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          3

In [9]:
datagen = ImageDataGenerator(rotation_range = 10, zoom_range = 0.1, width_shift_range = 0.1, height_shift_range = 0.1,)

train_batch = datagen.flow(x, y, batch_size = 64)
val_batch = datagen.flow(x_test, y_test, batch_size = 64)

### Defining the reduction rate if there is no significant improvement

In [10]:
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.1, min_lr = 0.00001)

In [11]:
history = model.fit(train_batch,
                              epochs = 5, 
                              steps_per_epoch = len(train_batch),
                              validation_data = val_batch,
                              validation_steps = len(val_batch),
                              verbose = 1,
                             callbacks = [learning_rate_reduction])

Epoch 1/5
1750/1750 [==============================] - 100s 56ms/step - loss: 0.3761 - accuracy: 0.9079 - val_loss: 0.2318 - val_accuracy: 0.9630 - lr: 0.0100
Epoch 2/5
1750/1750 [==============================] - 92s 53ms/step - loss: 0.2408 - accuracy: 0.9548 - val_loss: 0.1715 - val_accuracy: 0.9731 - lr: 0.0100
Epoch 3/5
1750/1750 [==============================] - 92s 53ms/step - loss: 0.2663 - accuracy: 0.9557 - val_loss: 0.1496 - val_accuracy: 0.9683 - lr: 0.0100
Epoch 4/5
1750/1750 [==============================] - 96s 55ms/step - loss: 0.3121 - accuracy: 0.9547 - val_loss: 0.1616 - val_accuracy: 0.9705 - lr: 0.0100
Epoch 5/5
1750/1750 [==============================] - 95s 54ms/step - loss: 0.3410 - accuracy: 0.9530 - val_loss: 0.7869 - val_accuracy: 0.9777 - lr: 0.0100


In [12]:
res = model.evaluate(x, y, batch_size = 1024)
print(res[1]*100)

110/110 [==============================] - 12s 105ms/step - loss: 6.2713 - accuracy: 0.9853
98.533034324646


### Saving the model

In [13]:
model.save("model2")

INFO:tensorflow:Assets written to: model2\assets
